In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import json

In [2]:
a = []
# b = []
matrix = []
heatmapMatrix = []

locList = ["Grocery Stores, Supermarkets", "Bakeries", "Fast Food Restaurants", "Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances", "Drug Stores and Pharmacies", "Book Stores", "Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local"]

for j in range(len(locList)):
    prob = 1
    a = []
    b = []
    for i in range(len(locList)):
        if i != len(locList)-1:
            num = 0.16 * (i+1) *  np.random.rand()
            indprob = num * prob
            a.append(indprob)
            heatmapMatrix.append([locList[i], locList[j], indprob])
            prob = prob - indprob
        else:
            a.append(prob)
            heatmapMatrix.append([locList[i], locList[j], prob])
    matrix.append(a)
    # heatmapMatrix.append(b)

# print(heatmapMatrix)
probMatrix = pd.DataFrame(matrix, columns = locList, index = locList)
heatmapProbMatrix = pd.DataFrame(heatmapMatrix, columns=["place1", "place2", "prob"])

In [3]:
probMatrix.head()

,"Grocery Stores, Supermarkets",Bakeries,Fast Food Restaurants,"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",Drug Stores and Pharmacies,Book Stores,"Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local"
"Grocery Stores, Supermarkets",0.146510,0.026219,0.239183,0.304300,0.194802,0.063693,0.025293
Bakeries,0.133234,0.202003,0.109147,0.184090,0.286933,0.002049,0.082543
Fast Food Restaurants,0.058849,0.189936,0.021357,0.369663,0.269088,0.043149,0.047959
"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",0.051163,0.125401,0.340064,0.281106,0.010568,0.005814,0.185882
Drug Stores and Pharmacies,0.028911,0.048461,0.421750,0.134831,0.057271,0.021479,0.287296


In [4]:
heatmapProbMatrix.to_csv("heatmapProbMatrix.csv")

In [5]:
pd.Timestamp("2020-04-30 09:00:00") + pd.offsets.Minute(10)  

Timestamp('2020-04-30 09:10:00')

In [12]:
%%time
allRec = []
routeLenth = 8
start_time = pd.Timestamp("2020-04-30")

# for j in range(routeLenth-1):
# #     rec = []
#     name = "people"+str(j)
pplCount = 0
for j in range(300): 
    for num, location in zip(range(len(locList)), locList):
        name = "people"+str(pplCount)
        station = location
        rec = []
        time = pd.Timestamp("2020-04-30 09:00:00") 
        rec.append(name)#name
        rec.append(station)#location
        rec.append(time) #time
        rec.append(np.random.randint(10, high=80)) #transaction val
        allRec.append(rec)
        pplCount += 1
        for i in range(routeLenth-1):
            rec = []
            time += pd.offsets.Minute(np.random.randint(1, high=200))  
            # print(station)
            probList = probMatrix.loc[station].to_list()
            station = np.random.choice(locList, 1, p=probList)[0]
            rec.append(name)#name
            rec.append(station)#location
            rec.append(time) #time
            rec.append(np.random.randint(10, high=80)) #transaction val
            allRec.append(rec)



CPU times: user 3.14 s, sys: 21.2 ms, total: 3.16 s
Wall time: 3.15 s


In [13]:
records = pd.DataFrame(allRec, columns = ["name", "mcc", "time", "transaction_value"])

In [14]:
records.head()

,name,mcc,time,transaction_value
0,people0,"Grocery Stores, Supermarkets",2020-04-30 09:00:00,29
1,people0,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 10:34:00,61
2,people0,Fast Food Restaurants,2020-04-30 10:37:00,67
3,people0,Drug Stores and Pharmacies,2020-04-30 12:24:00,29
4,people0,Fast Food Restaurants,2020-04-30 13:18:00,43


In [15]:
len(records)

16800

In [16]:
pd.options.mode.chained_assignment = None

In [17]:
%%time
records["name"] = records["name"].astype("category")
records["mcc"] = records["mcc"].astype("category")
nameList = records["name"].unique()
record_with_tinterval = pd.DataFrame([], columns = ["name", "mcc", "time", "transaction_value", "time_interval_to_next", "time_interval_from_last"])
#calculate time interval
for name in nameList:
    record_by_name = records[records["name"] == name]
    record_by_name["time_interval_to_next"] = -record_by_name["time"].diff(periods = -1)
    record_by_name["time_interval_from_last"] = record_by_name["time"].diff(periods = 1)
    record_by_name.replace(pd.Timedelta('nat'), pd.Timedelta(0), inplace=True)
    
    record_with_tinterval = record_with_tinterval.append(record_by_name)
#     print(record_by_name.head(2), record_with_tinterval.head())

record_with_tinterval.head()

CPU times: user 7.78 s, sys: 10.3 ms, total: 7.79 s
Wall time: 7.79 s


,name,mcc,time,transaction_value,time_interval_to_next,time_interval_from_last
0,people0,"Grocery Stores, Supermarkets",2020-04-30 09:00:00,29,01:34:00,00:00:00
1,people0,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 10:34:00,61,00:03:00,01:34:00
2,people0,Fast Food Restaurants,2020-04-30 10:37:00,67,01:47:00,00:03:00
3,people0,Drug Stores and Pharmacies,2020-04-30 12:24:00,29,00:54:00,01:47:00
4,people0,Fast Food Restaurants,2020-04-30 13:18:00,43,01:06:00,00:54:00


In [18]:
record_with_tinterval.to_csv("data_example_short.csv")

# Here is the main code

In [16]:
%%time
#read data and change data type
record_with_tinterval = pd.read_csv("data_example.csv", index_col = 0)

# record_with_tinterval["name"] = record_with_tinterval["name"].astype("category")
record_with_tinterval["mcc"] = record_with_tinterval["mcc"].astype("category")
record_with_tinterval["time"] =  pd.to_datetime(record_with_tinterval["time"])
record_with_tinterval["time_interval_to_next"] =  pd.to_timedelta(record_with_tinterval["time_interval_to_next"])
record_with_tinterval["time_interval_from_last"] =  pd.to_timedelta(record_with_tinterval["time_interval_from_last"])


CPU times: user 455 ms, sys: 12.5 ms, total: 468 ms
Wall time: 466 ms


In [17]:
record_with_tinterval.dtypes

name                                object
mcc                               category
time                        datetime64[ns]
transaction_value                    int64
time_interval_to_next      timedelta64[ns]
time_interval_from_last    timedelta64[ns]
dtype: object

In [18]:
record_with_tinterval.tail()

,name,mcc,time,transaction_value,time_interval_to_next,time_interval_from_last
76995,people6999,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 11:30:00,59,00:11:00,00:08:00
76996,people6999,Bakeries,2020-04-30 11:41:00,54,00:14:00,00:11:00
76997,people6999,Drug Stores and Pharmacies,2020-04-30 11:55:00,28,01:05:00,00:14:00
76998,people6999,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 13:00:00,64,00:23:00,01:05:00
76999,people6999,Drug Stores and Pharmacies,2020-04-30 13:23:00,23,00:00:00,00:23:00


In [19]:
#apply time filter: start time -> end time
startTime = pd.Timestamp("2020-04-30 10:00:00")
endTime = pd.Timestamp("2020-04-30 18:00:00")
test_time_range_filtered = record_with_tinterval[(record_with_tinterval["time"] > startTime)
                      & (record_with_tinterval["time"] < endTime)]

In [20]:
%%time
#like SQL's row_numebr, give a rank as time

test_time_range_filtered["step"] = test_time_range_filtered.groupby(["name"])["time"].rank().astype(int)

CPU times: user 14.6 ms, sys: 884 µs, total: 15.5 ms
Wall time: 14.4 ms


In [21]:
test_time_range_filtered.head()

,name,mcc,time,transaction_value,time_interval_to_next,time_interval_from_last,step
1,people0,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 11:20:00,71,00:02:00,02:20:00,1
2,people0,Fast Food Restaurants,2020-04-30 11:22:00,79,00:32:00,00:02:00,2
3,people0,Book Stores,2020-04-30 11:54:00,28,02:14:00,00:32:00,3
4,people0,"Motor Freight Carriers, Moving and Storage Com...",2020-04-30 14:08:00,48,00:09:00,02:14:00,4
5,people0,Drug Stores and Pharmacies,2020-04-30 14:17:00,35,02:31:00,00:09:00,5


In [22]:
len(test_time_range_filtered)

32944

In [23]:
%%time
queryMCC = "Bakeries"
#get largest min "max first position" of the queried MCC in the route as the left alignment position
test_data_query = test_time_range_filtered.loc[test_time_range_filtered["mcc"] == queryMCC]
grouped_test_data_query = test_data_query.groupby("name")
left_alignment_pos = grouped_test_data_query["step"].min().max()
need_alignment = grouped_test_data_query["step"].min().dropna() < left_alignment_pos
alignment_offset = left_alignment_pos - grouped_test_data_query["step"].min().dropna()
len(need_alignment)

CPU times: user 8.65 ms, sys: 1.08 ms, total: 9.73 ms
Wall time: 8.68 ms


3843

In [24]:
%%time
test_data_query_route = test_time_range_filtered[test_time_range_filtered["name"].isin(need_alignment.index)]
test_data_query_route.head()

CPU times: user 4.44 ms, sys: 1.09 ms, total: 5.53 ms
Wall time: 4.41 ms


,name,mcc,time,transaction_value,time_interval_to_next,time_interval_from_last,step
12,people1,Fast Food Restaurants,2020-04-30 12:04:00,75,01:46:00,03:04:00,1
13,people1,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 13:50:00,45,00:24:00,01:46:00,2
14,people1,Bakeries,2020-04-30 14:14:00,13,01:35:00,00:24:00,3
15,people1,Drug Stores and Pharmacies,2020-04-30 15:49:00,24,01:33:00,01:35:00,4
16,people1,Bakeries,2020-04-30 17:22:00,79,02:41:00,01:33:00,5


In [25]:
len(test_data_query_route)

18991

In [26]:
%%time
#move the order acoording to offset
test_data_query_route.set_index("name", inplace = True)
test_data_query_route["offset"] = alignment_offset
test_data_query_route["step"] = test_data_query_route["step"] + test_data_query_route["offset"]
test_data_query_route.head()

CPU times: user 4.34 ms, sys: 757 µs, total: 5.1 ms
Wall time: 4.3 ms


,mcc,time,transaction_value,time_interval_to_next,time_interval_from_last,step,offset
name,,,,,,,
people1,Fast Food Restaurants,2020-04-30 12:04:00,75,01:46:00,03:04:00,8,7
people1,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 13:50:00,45,00:24:00,01:46:00,9,7
people1,Bakeries,2020-04-30 14:14:00,13,01:35:00,00:24:00,10,7
people1,Drug Stores and Pharmacies,2020-04-30 15:49:00,24,01:33:00,01:35:00,11,7
people1,Bakeries,2020-04-30 17:22:00,79,02:41:00,01:33:00,12,7


In [64]:
%%time
columns = ["mcc", "transaction_value", "time_interval_to_next", "time_interval_from_last"]
shaped_routes = test_data_query_route.pivot(index = test_data_query_route.index, columns =  "step")
shaped_routes.head()

CPU times: user 31.1 ms, sys: 1.08 ms, total: 32.2 ms
Wall time: 31 ms


mcc                                \
step         1    2    3    4    5    6    7    
name                                            
people1     NaN  NaN  NaN  NaN  NaN  NaN  NaN   
people10    NaN  NaN  NaN  NaN  NaN  NaN  NaN   
people100   NaN  NaN  NaN  NaN  NaN  NaN  NaN   
people1000  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
people1001  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

                                                               \
step                                                       8    
name                                                            
people1                                 Fast Food Restaurants   
people10                                                  NaN   
people100   Furniture, Home Furnishings, and Equipment Sto...   
people1000                                                NaN   
people1001                                                NaN   

                                                                         ...  \
step                                                       9         10  ...   
name                                                                     ...   
people1     Furniture, Home Furnishings, and Equipment Sto...  Bakeries  ...   
people10                                Fast Food Restaurants  Bakeries  ...   
people100                               Fast Food Restaurants  Bakeries  ...   
people1000                                                NaN  Bakeries  ...   
people1001                                                NaN  Bakeries  ...   

           offset                                          
step           10   11   12   13   14  15  16  17  18  19  
name                                                       
people1       7.0  7.0  7.0  NaN  NaN NaN NaN NaN NaN NaN  
people10      8.0  8.0  8.0  8.0  8.0 NaN NaN NaN NaN NaN  
people100     7.0  7.0  7.0  NaN  NaN NaN NaN NaN NaN NaN  
people1000    9.0  9.0  9.0  NaN  NaN NaN NaN NaN NaN NaN  
people1001    9.0  9.0  9.0  NaN  NaN NaN NaN NaN NaN NaN  

[5 rows x 114 columns]

In [140]:
%%time
#mcc routes for calculate links

time_interval_limit = pd.offsets.Minute(100)
#calculate absolute step pos
single_sequence = 2
raw_step = single_sequence + left_alignment_pos
#last step targets
sample_last_step_list = ["Bakeries", "Book Stores", "Drug Stores and Pharmacies"]
keep_rank_num = 5

if single_sequence > 0:
    source_step = raw_step-1
    target_step = raw_step
    #filter the data by time interval
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][source_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][target_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
elif single_sequence < 0:
    source_step = raw_step+1
    target_step = raw_step
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][target_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][source_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})

#count the frequency of an OD
links_g = links.reset_index().groupby(["source","target"]).count().reset_index().dropna().rename(columns = {"name":"count"})
links_g["route"] = links.groupby(["source","target"]).groups.values()
links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
links_g["atv"] = links.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]
links_result = links_g[links_g["source"].isin(sample_last_step_list)].nlargest(keep_rank_num, "count").reset_index(drop=True)

# links["key"] = links["source"].astype(str)+links["target"].astype(str)
# links_result["key"] = links_result["source"].astype(str)+links_result["target"].astype(str)

links["key"] = list(zip(links["source"],links["target"]))
links_result_zip_key = list(itertools.product(sample_last_step_list,links_result["target"]))

nodes_g = links[links["key"].isin(links_result_zip_key)].drop(columns = ["source"]).groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
nodes_g["route"] = links[links["key"].isin(links_result_zip_key)].drop(columns = ["source"]).groupby(["target"]).groups.values()
# nodes_g["count"] = nodes_g["route"].apply(lambda x: len(x))
nodes_result = nodes_g[nodes_g["target"].isin(links_result["target"])]
#re-select links from link-node results, avoiding there is no links betweens nodes in different steps
links_g["key"] = list(zip(links_g["source"],links_g["target"]))
links_result = links_g[links_g["key"].isin(links_result_zip_key)].reset_index(drop=True)

links_result.drop(columns = "key", inplace = True)
nodes_result.drop(columns = "key", inplace = True)
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())

link_result_dict = links_result.to_dict('records')
nodes_result_dict = nodes_result.to_dict('records')
links_result
# links_result.drop["source"].groupby("target")
# nodes_g.groupby(["target"]).indices.values()

CPU times: user 49.9 ms, sys: 1.08 ms, total: 51 ms
Wall time: 49.9 ms


step,source,target,count,route,atv
0,Bakeries,Bakeries,28.0,"[people1056, people1235, people2226, people231...",47.571429
1,Bakeries,Drug Stores and Pharmacies,80.0,"[people1051, people1189, people1240, people133...",44.875000
2,Bakeries,Fast Food Restaurants,16.0,"[people1752, people2062, people2075, people256...",46.500000
3,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",21.0,"[people1459, people1614, people2146, people233...",43.380952
4,Book Stores,Fast Food Restaurants,6.0,"[people2816, people2868, people3118, people327...",46.666667
5,Book Stores,"Furniture, Home Furnishings, and Equipment Sto...",1.0,[people2001],34.000000
6,Drug Stores and Pharmacies,Bakeries,136.0,"[people1, people1022, people1055, people1058, ...",46.838235
7,Drug Stores and Pharmacies,Drug Stores and Pharmacies,181.0,"[people1043, people1157, people1181, people127...",45.629834
8,Drug Stores and Pharmacies,Fast Food Restaurants,185.0,"[people1004, people1040, people1064, people109...",45.264865
9,Drug Stores and Pharmacies,"Furniture, Home Furnishings, and Equipment Sto...",68.0,"[people10, people1011, people1053, people1204,...",45.264706


In [141]:
%%time
this_step_routes = []
for routes in links_result["route"]:
    this_step_routes += routes
# this_step_routes
links[links.index.isin(this_step_routes)]

CPU times: user 1.64 ms, sys: 307 µs, total: 1.94 ms
Wall time: 1.67 ms


step,source,target,transaction_value,key
name,,,,
people1,Drug Stores and Pharmacies,Bakeries,79.0,"(Drug Stores and Pharmacies, Bakeries)"
people10,Drug Stores and Pharmacies,"Furniture, Home Furnishings, and Equipment Sto...",36.0,"(Drug Stores and Pharmacies, Furniture, Home F..."
people1004,Drug Stores and Pharmacies,Fast Food Restaurants,60.0,"(Drug Stores and Pharmacies, Fast Food Restaur..."
people1011,Drug Stores and Pharmacies,"Furniture, Home Furnishings, and Equipment Sto...",52.0,"(Drug Stores and Pharmacies, Furniture, Home F..."
people1022,Drug Stores and Pharmacies,Bakeries,23.0,"(Drug Stores and Pharmacies, Bakeries)"
...,...,...,...,...
people977,Drug Stores and Pharmacies,Fast Food Restaurants,53.0,"(Drug Stores and Pharmacies, Fast Food Restaur..."
people989,Drug Stores and Pharmacies,Fast Food Restaurants,73.0,"(Drug Stores and Pharmacies, Fast Food Restaur..."
people990,Drug Stores and Pharmacies,Bakeries,37.0,"(Drug Stores and Pharmacies, Bakeries)"


In [144]:
%%time
#mcc routes for calculate links

time_interval_limit = pd.offsets.Minute(100)
#calculate absolute step pos
single_sequence = 4
raw_step = single_sequence + left_alignment_pos
#last step targets
# sample_last_step_list = ["Bakeries", "Book Stores", "Drug Stores and Pharmacies"]
keep_rank_num = 5

if single_sequence > 0:
    source_step = raw_step-1
    target_step = raw_step
    #filter the data by time interval
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][source_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][target_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
elif single_sequence < 0:
    source_step = raw_step+1
    target_step = raw_step
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][target_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][source_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})

#filter links into last step routes, then all the sources should be same as last steps' target
if abs(single_sequence) > 1:
    links = links[links.index.isin(this_step_routes)]
    
# add transaction value column
links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
# get nodes grouped
nodes_g = links.drop(columns = ["source"]).groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
# capture nodes' route
nodes_g["route"] = links.drop(columns = ["source"]).groupby(["target"]).groups.values()
# get top nodes as nodes' result
nodes_result = nodes_g.nlargest(keep_rank_num, "count").reset_index(drop=True)
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())
# get routes of top nodes 
this_step_routes = []
for routes in nodes_result["route"]:
    this_step_routes += routes
# get all links in this step's route
links = links[links.index.isin(this_step_routes)]
# get links grouped into source + target + count format df
links_g = links.reset_index().groupby(["source","target"]).count().reset_index().dropna().rename(columns = {"name":"count"})
# capture links' routes by people's identifier/name
links_g["route"] = links.groupby(["source","target"]).groups.values()
# add atv column
links_g["atv"] = links.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]
# generate results
links_result = links_g.reset_index(drop = True)
# convert route to list
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())

link_result_dict = links_result.to_dict('records')
nodes_result_dict = nodes_result.to_dict('records')
links_result
# links_result.drop["source"].groupby("target")
# nodes_g.groupby(["target"]).indices.values()

CPU times: user 42.2 ms, sys: 1.06 ms, total: 43.2 ms
Wall time: 42.2 ms


step,source,target,count,transaction_value,route,atv
0,Bakeries,Bakeries,4.0,4.0,"[people3985, people5452, people683, people729]",63.500000
1,Bakeries,Drug Stores and Pharmacies,9.0,9.0,"[people3242, people4192, people5336, people560...",40.555556
2,Bakeries,Fast Food Restaurants,5.0,5.0,"[people1318, people1798, people3204, people329...",44.600000
3,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",1.0,1.0,[people1292],57.000000
4,Bakeries,"Motor Freight Carriers, Moving and Storage Com...",4.0,4.0,"[people197, people2471, people4951, people5440]",34.750000
5,Drug Stores and Pharmacies,Bakeries,9.0,9.0,"[people1204, people1614, people2837, people324...",49.222222
6,Drug Stores and Pharmacies,Drug Stores and Pharmacies,9.0,9.0,"[people1460, people1537, people2406, people267...",40.666667
7,Drug Stores and Pharmacies,Fast Food Restaurants,10.0,10.0,"[people1997, people238, people3225, people331,...",47.600000
8,Drug Stores and Pharmacies,"Furniture, Home Furnishings, and Equipment Sto...",7.0,7.0,"[people1235, people135, people2857, people3236...",55.285714
9,Drug Stores and Pharmacies,"Motor Freight Carriers, Moving and Storage Com...",3.0,3.0,"[people1270, people1335, people2226]",35.333333


In [90]:

links["key"] = list(zip(links["source"],links["target"]))
links_result["key"] = list(zip(links_result["source"],links_result["target"]))
nodes_g = links[links["key"].isin(links_result["key"])].drop(columns = ["source"]).groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
nodes_g["route"] = links[links["key"].isin(links_result["key"])].drop(columns = ["source"]).groupby(["target"]).groups.values()
# nodes_g["count"] = nodes_g["route"].apply(lambda x: len(x))
nodes_result = nodes_g[nodes_g["target"].isin(links_result["target"])]


In [143]:
nodes_result

step,target,count,route
0,Drug Stores and Pharmacies,85,"[people1051, people1056, people1145, people119..."
1,Bakeries,56,"[people1055, people1292, people1318, people179..."
2,Fast Food Restaurants,53,"[people1022, people1095, people1309, people159..."
3,"Grocery Stores, Supermarkets",42,"[people1990, people2052, people2062, people211..."
4,"Furniture, Home Furnishings, and Equipment Sto...",33,"[people10, people1058, people1240, people1423,..."


In [66]:
%%time
startTime = pd.Timestamp("2020-04-30 10:00:00")
endTime = pd.Timestamp("2020-04-30 18:00:00")
time_interval_limit = pd.offsets.Minute(100)
sample_last_step_list = ["Bakeries", "Book Stores"]

def QuerySingleNode(startTime, endTime, queryMCC, single_sequence, time_interval_limit, last_step_list, keep_rank_num):
    time_range_filtered = record_with_tinterval[(record_with_tinterval["time"] > startTime)
                          & (record_with_tinterval["time"] < endTime)]
    time_range_filtered["step"] = time_range_filtered.groupby(["name"])["time"].rank().astype(int)
#     queryMCC = "Bakeries"
    #get largest min "max first position" of the queried MCC in the route as the left alignment position
    data_mcc_query = time_range_filtered.loc[time_range_filtered["mcc"] == queryMCC]
    grouped_data_query = data_mcc_query.groupby("name")
    left_alignment_pos = grouped_data_query["step"].min().max()
    need_alignment = grouped_data_query["step"].min().dropna() < left_alignment_pos
    #calcu alignment offset
    alignment_offset = left_alignment_pos - grouped_data_query["step"].min().dropna()
    data_query_route = time_range_filtered[time_range_filtered["name"].isin(need_alignment.index)]
    data_query_route.set_index("name", inplace = True)
    data_query_route["offset"] = alignment_offset
    data_query_route["step"] = data_query_route["step"] + data_query_route["offset"]
    columns = ["mcc", "transaction_value", "time_interval_to_next", "time_interval_from_last"]
    shaped_routes = data_query_route.pivot(index = data_query_route.index, columns =  "step")
#     time_interval_limit = pd.offsets.Minute(100)
    #calculate absolute step pos
#     single_sequence = 2
    raw_step = single_sequence + left_alignment_pos
    #last step targets
#     sample_last_step_list = ["Bakeries", "Book Stores"]
#     keep_rank_num = 5

    if single_sequence > 0:
        source_step = raw_step-1
        target_step = raw_step
        #filter the data by time interval
        filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][source_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][target_step] < time_interval_limit)]
        links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
    elif single_sequence < 0:
        source_step = raw_step+1
        target_step = raw_step
        filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][target_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][source_step] < time_interval_limit)]
        links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})

    #count the frequency of an OD
    links_g = links.reset_index().groupby(["source","target"]).count().reset_index().dropna().rename(columns = {"name":"count"})
    links_g["route"] = links.groupby(["source","target"]).groups.values()
    links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
    links_g["atv"] = links.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]
    links_result = links_g[links_g["source"].isin(sample_last_step_list)].nlargest(keep_rank_num, "count").reset_index(drop=True)

    links["key"] = links["source"].astype(str)+links["target"].astype(str)
    links_result["key"] = links_result["source"].astype(str)+links_result["target"].astype(str)

    nodes_g = links[links["key"].isin(links_result["key"])].drop(columns = ["source"]).groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
    nodes_g["route"] = links[links["key"].isin(links_result["key"])].drop(columns = ["source"]).groupby(["target"]).groups.values()
    # nodes_g["count"] = nodes_g["route"].apply(lambda x: len(x))
    nodes_result = nodes_g[nodes_g["target"].isin(links_result["target"])]

    links_result.drop(columns = "key", inplace = True)
    nodes_result.drop(columns = "key", inplace = True)
    links_result["route"] = links_result["route"].apply(lambda x: x.tolist())
    nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())

    link_result_dict = links_result.to_dict('records')
    nodes_result_dict = nodes_result.to_dict('records')
    
    return links_result



CPU times: user 42 µs, sys: 1 µs, total: 43 µs
Wall time: 46.3 µs


In [67]:
%%time
startTime = pd.Timestamp("2020-04-30 10:00:00")
endTime = pd.Timestamp("2020-04-30 18:00:00")
time_interval_limit = pd.offsets.Minute(100)
sample_last_step_list = ["Bakeries", "Book Stores"]
QuerySingleNode(startTime, endTime, "Bakeries", -2, time_interval_limit, sample_last_step_list, 5)

CPU times: user 96.9 ms, sys: 2.49 ms, total: 99.4 ms
Wall time: 98.4 ms


step,source,target,count,route,atv
0,Book Stores,Fast Food Restaurants,11.0,"[people2587, people3052, people3135, people377...",39.363636
1,Book Stores,Book Stores,6.0,"[people1616, people3649, people4895, people513...",43.833333
2,Book Stores,"Motor Freight Carriers, Moving and Storage Com...",5.0,"[people2132, people3056, people342, people384,...",35.800000
3,Book Stores,Drug Stores and Pharmacies,1.0,[people6197],46.000000
4,Book Stores,"Grocery Stores, Supermarkets",1.0,[people1427],55.000000


In [29]:
tst_g.groupby("target").mean()

NameError: name 'tst_g' is not defined

In [ ]:
%%time
#all mcc mean
test_time_range_filtered["transaction_value"] = test_time_range_filtered["transaction_value"].astype(float)
test_time_range_filtered.groupby("mcc", sort=False)["transaction_value"].mean()